#### Imports

In [1]:
import requests
from requests import get
from bs4 import BeautifulSoup
import pandas as pd

import os
from pprint import pprint 
import time
import json

import acquire

In [2]:
#standard imports
import numpy as np

#import
import unicodedata
#import regular expression operations
import re
#import natural language toolkit
import nltk
#import stopwords list
from nltk.corpus import stopwords


In [3]:
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
from wordcloud import WordCloud

#### Acquire

In [ ]:
# url = 'https://github.com/search?o=desc&q=sports+betting&s=stars&type=Repositories&p=1'

# response = get(url)
# response

In [ ]:
# soup = BeautifulSoup(response.content, 'html.parser')
# soup

In [ ]:
# repos = soup.find_all('a', class_="v-align-middle")
# repos

In [ ]:
# extract the links into a list
# link_list = [link['href'] for link in repos]
# link_list

In [ ]:
# ['https://github.com'+link['href'] for link in repos]

In [ ]:
# url = 'https://github.com/search?o=desc&q=sports+betting&s=stars&type=Repositories&p='

In [ ]:
# link_list = []

# for x in range(1,13):
#     # print(x)
#     print(url+str(x))
#     response = get(url+str(x))
#     soup = BeautifulSoup(response.content, 'html.parser')
#     repos = soup.find_all('a', class_="v-align-middle")
#     # print(repos)
#     time.sleep(10)
#     print(len(repos))
#     link_list.extend([link['href'] for link in repos])

In [ ]:
# len(link_list)

In [12]:
df = pd.read_json('data.json')

In [ ]:
# data = acquire.scrape_github_data()

In [ ]:
# json.dump(data, open("data.json", "w"), indent=1)

In [ ]:
# df = pd.DataFrame(data)

In [ ]:
df = df.dropna()

In [13]:

df.readme_contents = df.readme_contents.str.replace('[/,_,-,:,"]', ' ', regex=True)

df.readme_contents = df.readme_contents.str.replace('heavy', '').str.replace('check', '').str.replace('mark', '')

In [14]:
df.language.value_counts()

JavaScript           24
Python               23
Jupyter Notebook     10
HTML                  8
R                     8
TypeScript            6
Ruby                  5
PHP                   4
C#                    4
Solidity              2
Rust                  2
Visual Basic .NET     1
Go                    1
Swift                 1
Java                  1
Name: language, dtype: int64

In [ ]:
def basic_clean(original):
    '''
    Input: original text or .apply(basic_clean) to entire data frame
    Actions: 
    lowercase everything,
    normalizes everything,
    removes anything that's not a letter, number, whitespace, or single quote
    Output: Cleaned text
    '''
    # lowercase everything
    basic_cleaned = original.lower()
    # normalize unicode characters
    basic_cleaned = unicodedata.normalize('NFKD', basic_cleaned)\
    .encode('ascii', 'ignore')\
    .decode('utf-8')
    # Replace anything that is not a letter, number, whitespace or a single quote.
    basic_cleaned = re.sub(r'[^a-z0-9\'\s]', '', basic_cleaned)
    
    return basic_cleaned

In [ ]:
def tokenize(basic_cleaned):
    '''
    Input: basic_cleaned text string or .apply(tokenize) to entire data frame
    Actions:
    creates the tokenizer
    uses the tokenizer
    Output: clean_tokenize text string
    '''
    #create the tokenizer
    tokenize = nltk.tokenize.ToktokTokenizer()
    #use the tokenizer
    clean_tokenize = tokenize.tokenize(basic_cleaned, return_str=True)
    
    return clean_tokenize


In [ ]:
def remove_stopwords(lemma_or_stem, extra_words=[], exclude_words=[]):
    '''
    Input:text string or .apply(remove_stopwords) to entire data frame
    Action: removes standard stop words
    Output: parsed_article
    '''
    # save stopwords
    stopwords_ls = stopwords.words('english')
    # removing any stopwords in exclude list
    stopwords_ls = set(stopwords_ls) - set(exclude_words)
    # adding any stopwords in extra list
    stopwords_ls = stopwords_ls.union(set(extra_words))
    
    # split words in article
    words = lemma_or_stem.split()
    # remove stopwords from list of words
    filtered = [word for word in words if word not in stopwords_ls]
    # join words back together
    parsed_article = ' '.join(filtered)
    
    return parsed_article


In [ ]:
def lemmatize(clean_tokenize):
    '''
    Inputs: clean_tokenize
    Actions: creates lemmatizer and applies to each word
    Outputs: clean_tokenize_lemma
    '''
    #create the lemmatizer
    wnl = nltk.stem.WordNetLemmatizer()
    #use lemmatize - apply to each word in our string
    lemmas = [wnl.lemmatize(word) for word in clean_tokenize.split()]
    #join words back together
    clean_tokenize_lemma = ' '.join(lemmas)
    
    return clean_tokenize_lemma



In [ ]:
def clean(text):
    '''
    A simple function to cleanup text data.
    
    Args:
        text (str): The text to be cleaned.
        
    Returns:
        list: A list of lemmatized words after cleaning.
    '''
    
    # basic_clean() function from last lesson:
    # Normalize text by removing diacritics, encoding to ASCII, decoding to UTF-8, and converting to lowercase
    text = (unicodedata.normalize('NFKD', text)
             .encode('ascii', 'ignore')
             .decode('utf-8', 'ignore')
             .lower())
    
    # Remove punctuation, split text into words
    words = re.sub(r'[^\w\s]', '', text).split()
    
    
    # lemmatize() function from last lesson:
    # Initialize WordNet lemmatizer
    wnl = nltk.stem.WordNetLemmatizer()
    
    # Combine standard English stopwords with additional stopwords
    stopwords = nltk.corpus.stopwords.words('english')
    
    # Lemmatize words and remove stopwords
    cleaned_words = [wnl.lemmatize(word) for word in words if word not in stopwords]
    
    return cleaned_words

In [ ]:
# df = df.rename(columns={'readme_contents': 'original'})

In [ ]:

df['clean_norm_token'] = df.readme_contents.apply(tokenize).apply(basic_clean).apply(remove_stopwords).apply(lemmatize)


df.clean_norm_token = df.clean_norm_token.str.replace('124', '')

In [15]:
#in language column replace language with other if it is not in the top 3 languages
top_5 = df.language.value_counts().head(5).index.tolist()
print(top_5)
df.language = df.language.apply(lambda x: x if x in top_5 else 'other')

['JavaScript', 'Python', 'Jupyter Notebook', 'HTML', 'R']


In [ ]:
df.language.value_counts()

In [ ]:
#baseline is 43%
df.language.value_counts(normalize=True)

In [7]:
import wrangle as w
import auto_model2a1 as atm

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [16]:
#calls the split function from wrangle.py
train, validate, test = w.split_function_cat_target(df, "language")

In [17]:
df.head()

,repo,language,readme_contents
0,/kyleskom/NBA-Machine-Learning-Sports-Betting,Python,# NBA Sports Betting Using Machine Learning 🏀\...
1,/pretrehr/Sports-betting,Python,[![forthebadge made-with-python](http://ForThe...
2,/llSourcell/ChatGPT_Sports_Betting_Bot,Jupyter Notebook,# ChatGPT Sports Betting Bot\n\nThis is the co...
3,/georgedouzas/sports-betting,Python,[scikit-learn]: <http://scikit-learn.org/stabl...
4,/sedemmler/WagerBrain,Python,# WagerBrain\nA package containing the essenti...


In [18]:
train.head()

,repo,language,readme_contents
62,/valenIndovino/apuestas-deportivas,HTML,# ApuestasOficial\nDesarrollo del Proyecto: Pa...
13,/JDaniloC/BetBot,HTML,# BetBot\n\n![BetBot Video](./docs/images/vide...
26,/addtek/reactnative_sports_betting_app,JavaScript,# reactnative_sports_betting_app\nA Sport bett...
28,/mautomic/NitrogenSports-Analysis,Python,## NitrogenSports-Analysis\n\nThis script is m...
16,/dashee87/betScrapeR,R,"---\r\ntitle: ""README""\r\noutput:\r\n md_docu..."


In [ ]:
df.tail()

In [ ]:
python_words = w.clean(' '.join(train[train.language=='Python']['lemmatized']))
java_script_words = w.clean(' '.join(train[train.language=='JavaScript']['lemmatized']))
jupyter_notebook_words = w.clean(' '.join(train[train.language=='Jupyter Notebook']['lemmatized']))
html_words = w.clean(' '.join(train[train.language=='HTML']['lemmatized']))
r_words = w.clean(' '.join(train[train.language=='R']['lemmatized']))
other_words = w.clean(' '.join(train[train.language=='other']['lemmatized']))

all_words = w.clean(' '.join(df['lemmatized']))

In [ ]:
python_freq = pd.Series(python_words).value_counts()
java_script_freq = pd.Series(java_script_words).value_counts()
jupyter_notebook_freq = pd.Series(jupyter_notebook_words).value_counts()
html_freq = pd.Series(html_words).value_counts()
r_freq = pd.Series(r_words).value_counts()
other_freq = pd.Series(other_words).value_counts()

all_freq = pd.Series(all_words).value_counts()

In [ ]:
def make_ngrams(words, n):
    return pd.Series(nltk.ngrams(words, n)).value_counts().head(20)

In [ ]:
def plot_bigrams(words):
    word_data = {k[0] + ' ' + k[1]: v for k, v in words.to_dict().items()}
    
    word_img = WordCloud(background_color='white', width=800, height=400).generate_from_frequencies(word_data)
    
    plt.figure(figsize=(8, 4))
    plt.imshow(word_img)
    plt.axis('off')
    plt.show()

In [ ]:
# setting basic style parameters for matplotlib
plt.rc('figure', figsize=(13, 7))
plt.style.use('seaborn-darkgrid')

In [ ]:
# lets combine all 3 dfs horizontally
# fill any missing values with zero
# convert the resulting df to contain only integer values:

word_counts = pd.concat([python_freq, java_script_freq, jupyter_notebook_freq,
                        html_freq, r_freq, all_freq], axis=1).fillna(0).astype(int)

# rename the col names
word_counts.columns = ['python', 'java_script', 'jupyter_notebook', 'html', 'r', 'all']

word_counts.head()

In [ ]:
pd.Series(nltk.ngrams(python_words,2)).value_counts().head(20).plot.barh()

In [ ]:
blog_img = WordCloud(background_color='white').generate_from_frequencies(python_freq)
plt.imshow(blog_img)
plt.axis('off')
plt.show()

In [ ]:
# df.original = df.original.str.replace('[/,_,-,:,"]', ' ', regex=True)

In [ ]:
# pd.set_option("display.max_colwidth", None)

In [ ]:
# pd.reset_option("display.max_colwidth")

In [ ]:
# df.language.value_counts()

In [ ]:
blog_img = WordCloud(background_color='white').generate_from_frequencies(java_script_freq)
plt.imshow(blog_img)
plt.axis('off')
plt.show()

In [ ]:
#sets the X and y variables for train, validate, and test
X_train = train.clean_norm_token
y_train = train.language
X_validate = validate.clean_norm_token
y_validate = validate.language
X_test = test.clean_norm_token
y_test = test.language

In [ ]:
X_train.head()

In [ ]:
#make my bag of words Term Frequency 
cv = CountVectorizer()
X_bow = cv.fit_transform(X_train) 
X_validate_bow = cv.transform(X_validate)
X_test_bow = cv.transform(X_test)

In [ ]:
X_validate_bow.shape

In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

In [ ]:
#auto runs through knn models with k values 5-20
atm.auto_knn_scores(X_bow, y_train, X_validate_bow, y_validate)

In [ ]:
def k_nearest2(X_train_scaled, y_train, X_validate_scaled, y_validate):
    """
    Trains and evaluates KNN models for different values of k and plots the results.
    Parameters:
    -----------
    X_train: array-like, shape (n_samples, n_features)
        Training input samples.
    y_train: array-like, shape (n_samples,)
        Target values for the training input samples.
    X_validate: array-like, shape (n_samples, n_features)
        Validation input samples.
    y_validate: array-like, shape (n_samples,)
        Target values for the validation input samples.
    Returns:
    --------
    results: pandas DataFrame
        Contains the train and validation accuracy for each value of k.
    """
    metrics = []
    train_score = []
    validate_score = []
    for k in range(1,21):
        knn = KNeighborsClassifier(n_neighbors=k)
        knn.fit(X_train_scaled, y_train)
        train_score.append(knn.score(X_train_scaled, y_train))
        validate_score.append(knn.score(X_validate_scaled, y_validate))
        diff_score = train_score[-1] - validate_score[-1]
        metrics.append({'k': k, 'train_score': train_score[-1], 'validate_score': validate_score[-1], 'diff_score': diff_score})
    baseline_accuracy = y_train.value_counts().max() / y_train.value_counts().sum()
    results = pd.DataFrame.from_records(metrics)
    # modify the last few lines of the function
    # drop the diff_score column before plotting
    results_for_plotting = results.drop(columns=['diff_score'])
    with sns.axes_style('whitegrid'):
        ax = results_for_plotting.set_index('k').plot(figsize=(16,9))
    plt.ylabel('Accuracy')
    plt.axhline(baseline_accuracy, linewidth=2, color='black', label='baseline')
    plt.xticks(np.arange(0,21,1))
    min_diff_idx = np.abs(results['diff_score']).argmin()
    min_diff_k = results.loc[min_diff_idx, 'k']
    min_diff_score = results.loc[min_diff_idx, 'diff_score']
    ax.axvline(min_diff_k, linestyle='--', linewidth=2, color='red', label=f'min diff at k={min_diff_k} (diff={min_diff_score:.3f})')
    plt.fill_between(results['k'], train_score, validate_score, alpha=0.2, color='gray', where=(results['k'] > 0))
    plt.title('K Nearest Neighbor', fontsize=18)
    plt.legend()
    plt.show()
    
    return results


In [ ]:
y_train.value_counts().max() / y_train.value_counts().sum()

In [ ]:
k_nearest2(X_bow, y_train, X_validate_bow, y_validate)

In [ ]:
#auto runs through random forest models with max_depth values 5-20
atm.auto_random_forest_scores(X_bow, y_train, X_validate_bow, y_validate)

In [ ]:
def random_forest_scores(X_train_scaled, y_train, X_validate_scaled, y_validate):
    """
    Trains and evaluates a random forest classifier with different combinations of hyperparameters. The function takes in
    training and validation datasets, and returns a dataframe summarizing the model performance on each combination of
    hyperparameters.
    Parameters:
    -----------
    X_train : pandas DataFrame
        Features of the training dataset.
    y_train : pandas Series
        Target variable of the training dataset.
    X_validate : pandas DataFrame
        Features of the validation dataset.
    y_validate : pandas Series
        Target variable of the validation dataset.
    Returns:
    --------
    df : pandas DataFrame
        A dataframe summarizing the model performance on each combination of hyperparameters.
    """
    #define variables
    train_scores = []
    validate_scores = []
    min_samples_leaf_values = [1, 2, 3, 4, 5, 6, 7, 8 , 9, 10]
    max_depth_values = [10, 9, 8, 7, 6, 5, 4, 3, 2, 1]
    for min_samples_leaf, max_depth in zip(min_samples_leaf_values, max_depth_values):
        rf = RandomForestClassifier(min_samples_leaf=min_samples_leaf, max_depth=max_depth,random_state=123)
        rf.fit(X_train_scaled, y_train)
        train_score = rf.score(X_train_scaled, y_train)
        validate_score = rf.score(X_validate_scaled, y_validate)
        train_scores.append(train_score)
        validate_scores.append(validate_score)
    # Calculate the difference between the train and validation scores
    diff_scores = [train_score - validate_score for train_score, validate_score in zip(train_scores, validate_scores)]
    #Put results into a dataframe
    df = pd.DataFrame({
        'min_samples_leaf': min_samples_leaf_values,
        'max_depth': max_depth_values,
        'train_score': train_scores,
        'validate_score': validate_scores,
        'diff_score': diff_scores})
    # Set plot style
    sns.set_style('whitegrid')
    # Create plot
    plt.figure(figsize=(8, 6))
    plt.plot(max_depth_values, train_scores, label='train', marker='o', color='blue')
    plt.plot(max_depth_values, validate_scores, label='validation', marker='o', color='orange')
    plt.fill_between(max_depth_values, train_scores, validate_scores, alpha=0.2, color='gray')
    plt.xticks([2,4,6,8,10],['Leaf 9 and Depth 2','Leaf 7 and Depth 4','Leaf 5 and Depth 6','Leaf 3 and Depth 8','Leaf 1and Depth 10'], rotation = 45)
    plt.xlabel('min_samples_leaf and max_depth', fontsize=14)
    plt.ylabel('Accuracy', fontsize=14)
    plt.title('Random Forest Classifier Performance', fontsize=18)
    plt.legend(fontsize=12)
    plt.show()
    
    return df


In [ ]:
random_forest_scores(X_bow, y_train, X_validate_bow, y_validate)

In [ ]:
#auto runs through decision tree models with the "entropy" parameter with max_depth values 5-20
atm.auto_random_forest_entropy_scores(X_bow, y_train, X_validate_bow, y_validate)

In [ ]:
#auto runs through logistic regression models with C values .1-1
atm.auto_lo_regress_scores(X_bow, y_train, X_validate_bow, y_validate)

In [ ]:
#make my bag of words TF-IDF
tfidf = TfidfVectorizer()
X_bow = tfidf.fit_transform(X_train) 
X_validate_bow = tfidf.transform(X_validate)
X_test_bow = tfidf.transform(X_test)

In [ ]:
#auto runs through knn models with k values 5-20
atm.auto_knn_scores(X_bow, y_train, X_validate_bow, y_validate)

In [ ]:
#auto runs through random forest models with max_depth values 5-20
atm.auto_random_forest_scores(X_bow, y_train, X_validate_bow, y_validate)

In [ ]:
#auto runs through decision tree models with the "entropy" parameter with max_depth values 5-20
atm.auto_random_forest_entropy_scores(X_bow, y_train, X_validate_bow, y_validate)

In [ ]:
#auto runs through logistic regression models with C values .1-1
atm.auto_lo_regress_scores(X_bow, y_train, X_validate_bow, y_validate)